In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os
import sys
import webbrowser
from time import time_ns

sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
from interfaces.alchemy import AlchemyInterface
import configparser
from db.schema import StationLocation

In [ ]:
c_parser = configparser.ConfigParser()
c_parser.read("../../config.ini")
config = {"db": dict(c_parser["db"])}

In [ ]:
alchemy_interface = AlchemyInterface(config)

In [ ]:
time_start = time_ns()
data = alchemy_interface.select_obj(StationLocation)
time_query = time_ns()
print("Time to query data: ", (time_query - time_start) / 1e6, " ms")

In [ ]:
df = pd.DataFrame().from_records(
    [packet for packet in data], columns=StationLocation.__columns__
)

In [ ]:
df.info()

In [ ]:
df = df.dropna(subset=["latitude", "longitude"])

In [ ]:
coords = df[["latitude", "longitude"]].values

# Usar NearestNeighbors para encontrar vecinos en un radio
# El radio está en grados y puedes ajustarlo según tus necesidades
radius = 0.5  # Ajusta este valor según tus necesidades
nbrs = NearestNeighbors(radius=radius, algorithm="ball_tree").fit(coords)
distances, indices = nbrs.radius_neighbors(coords)

# Calcular la densidad: número de puntos en el radio para cada punto
density = np.array([len(indices[i]) for i in range(len(coords))])

# Agregar la densidad al DataFrame original
df["density"] = density

In [ ]:
def do_click(trace, points, state):
    print("clicked")
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.link.iloc[ind]
        webbrowser.open_new_tab(url)


# The logic of drawing a graph
def bar_func(df):
    fig = px.scatter_mapbox(
        df,
        lon="longitude",
        lat="latitude",
    )

    fig.update_traces(
        marker=go.scattermapbox.Marker(
            size=5,
        ),
    )

    fig.update_layout(
        hovermode="closest",
        mapbox_style="dark",
        mapbox_accesstoken=c_parser["mapbox"]["token"],
    )
    fig.data[0].on_click(do_click)
    fig.show()


bar_func(df)

In [28]:
x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])
scatter = f.data[0]
f.layout.hovermode = "closest"


def do_click(trace, points, state):
    print("Clicked")
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.link.iloc[ind]
        webbrowser.open_new_tab(url)


scatter.on_click(do_click)
f

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': 'a069bf5c-e50e-47c4-bdb6-f818417f26b7',
              'x': array([0.32580997, 0.88982734, 0.75170772, 0.7626321 , 0.46947903, 0.2107645 ,
                          0.04147508, 0.3218288 , 0.03711266, 0.69385541, 0.67035003, 0.43047178,
                          0.76778898, 0.53600849, 0.03985993, 0.13479312, 0.1934164 , 0.3356638 ,
                          0.05231295, 0.60511678, 0.51206103, 0.61746101, 0.43235559, 0.84770047,
                          0.45405906, 0.01540352, 0.87306815, 0.65620155, 0.82300304, 0.95177569,
                          0.05091238, 0.23507186, 0.06334344, 0.42165789, 0.86382915, 0.08162398,
                          0.47311195, 0.12554311, 0.7728856 , 0.84142216, 0.04329094, 0.48644074,
                          0.23941104, 0.95247378, 0.94389263, 0.613934  , 0.9734874 , 0.34486134,
                          0.89785072, 0.43459497, 0.23581464, 0.94082796, 

Clicked


IndexError: single positional indexer is out-of-bounds

In [29]:
import plotly.graph_objects as go

import numpy as np

np.random.seed(1)

x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode="markers")])

scatter = f.data[0]
colors = ["#a3a7e4"] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = "closest"


# create our callback function
def update_point(trace, points, selector):
    print("Hemlo")
    # c = list(scatter.marker.color)
    # s = list(scatter.marker.size)
    # for i in points.point_inds:
    #     c[i] = '#bae2be'
    #     s[i] = 20
    #     with f.batch_update():
    #         scatter.marker.color = c
    #         scatter.marker.size = s


scatter.on_click(update_point)

f

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                         

Hemlo
Hemlo
